# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger','stopwords'])
import pandas as pd 
import numpy as np
import re
import pickle
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import f1_score, classification_report, accuracy_score, make_scorer
from scipy.stats.mstats import gmean
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\varez\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\varez\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\varez\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\varez\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///../data/DisasterResponse.db')
df = pd.read_sql_table('messages',engine)
X = df['message']
Y = df.drop(['message','genre','original','id'],axis=1)

In [3]:
Y.columns

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    text = re.sub('[^a-zA-Z0-9]',' ',text)
    words = word_tokenize(text)
    words = [w for w in words if w not in stopwords.words('english')]
    
    lemmatizer = WordNetLemmatizer()
    clean_words = []
    for word in words: 
        clean_word = lemmatizer.lemmatize(word).lower().strip()
        clean_words.append(clean_word)
    return clean_words

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
    ('vect',CountVectorizer(tokenizer=tokenize)),
    ('tfidf',TfidfTransformer()),
    ('clf',MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X,Y)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                                                                        ccp_alpha=0.0,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                   

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
y_pred = pipeline.predict(X_test)

In [8]:
category_names = Y.columns
for i in range(len(category_names)):
    print('-'*60,'\n',"Category:", category_names[i],"\n", classification_report(y_test.iloc[:, i].values, y_pred[:, i]))
    print('Accuracy of',category_names[i], accuracy_score(y_test.iloc[:, i].values, y_pred[:,i]))


------------------------------------------------------------ 
 Category: related 
               precision    recall  f1-score   support

           0       0.71      0.46      0.56      1538
           1       0.85      0.93      0.89      4974
           2       0.25      0.40      0.31        42

    accuracy                           0.82      6554
   macro avg       0.60      0.60      0.59      6554
weighted avg       0.81      0.82      0.81      6554

Accuracy of related 0.8199572779981691
------------------------------------------------------------ 
 Category: request 
               precision    recall  f1-score   support

           0       0.90      0.98      0.94      5448
           1       0.83      0.48      0.61      1106

    accuracy                           0.90      6554
   macro avg       0.87      0.73      0.78      6554
weighted avg       0.89      0.90      0.88      6554

Accuracy of request 0.896551724137931
-------------------------------------------------

------------------------------------------------------------ 
 Category: hospitals 
               precision    recall  f1-score   support

           0       0.99      1.00      0.99      6480
           1       0.00      0.00      0.00        74

    accuracy                           0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.98      0.99      0.98      6554

Accuracy of hospitals 0.9887091852303936
------------------------------------------------------------ 
 Category: shops 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      6522
           1       0.00      0.00      0.00        32

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      1.00      0.99      6554

Accuracy of shops 0.9951174855050351
------------------------------------------------------------ 
 Category: aid_centers 
               p

In [9]:
def multioutput_f1score(y_true, y_pred): 
    scores = []
    for i in range(Y.shape[1]): 
        score = f1_score(y_true.iloc[:,i], y_pred[:,i],average='weighted')
        scores.append(score)
    scores = np.asarray(scores)
    score = gmean(scores)
    return score

In [10]:
multioutput_f1score(y_test,y_pred)

0.9351095314381396

### 6. Improve your model
Use grid search to find better parameters. 

In [11]:
parameters = {
    'clf__estimator__n_estimators':[100,200],
    'clf__estimator__min_samples_split':[2,3,4],
    'clf__estimator__criterion': ['entropy', 'gini']
}

scorer = make_scorer(multioutput_f1score,greater_is_better = True)

cv = GridSearchCV(pipeline,param_grid=parameters,verbose = 2, n_jobs = -1)
cv.fit(X_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 266.1min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed: 487.4min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        p

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [12]:
cv.best_estimator_

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                                                                        ccp_alpha=0.0,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                   

In [13]:
y_pred = cv.best_estimator_.predict(X_test)

multioutput_f1score(y_test,y_pred)

0.9356614164088237

In [14]:
for i in range(len(category_names)):
    print('-'*60,'\n',"Category:", category_names[i],"\n", classification_report(y_test.iloc[:, i].values, y_pred[:, i]))
    print('Accuracy of',category_names[i], accuracy_score(y_test.iloc[:, i].values, y_pred[:,i]))

------------------------------------------------------------ 
 Category: related 
               precision    recall  f1-score   support

           0       0.71      0.44      0.55      1538
           1       0.84      0.94      0.89      4974
           2       0.23      0.33      0.27        42

    accuracy                           0.82      6554
   macro avg       0.59      0.57      0.57      6554
weighted avg       0.81      0.82      0.80      6554

Accuracy of related 0.8187366493744278
------------------------------------------------------------ 
 Category: request 
               precision    recall  f1-score   support

           0       0.91      0.98      0.94      5448
           1       0.83      0.50      0.62      1106

    accuracy                           0.90      6554
   macro avg       0.87      0.74      0.78      6554
weighted avg       0.89      0.90      0.89      6554

Accuracy of request 0.8977723527616722
------------------------------------------------

------------------------------------------------------------ 
 Category: transport 
               precision    recall  f1-score   support

           0       0.96      1.00      0.98      6261
           1       0.81      0.07      0.13       293

    accuracy                           0.96      6554
   macro avg       0.88      0.54      0.56      6554
weighted avg       0.95      0.96      0.94      6554

Accuracy of transport 0.9577357339029601
------------------------------------------------------------ 
 Category: buildings 
               precision    recall  f1-score   support

           0       0.96      1.00      0.98      6231
           1       0.76      0.12      0.21       323

    accuracy                           0.95      6554
   macro avg       0.86      0.56      0.59      6554
weighted avg       0.95      0.95      0.94      6554

Accuracy of buildings 0.9548367409215746
------------------------------------------------------------ 
 Category: electricity 
        

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [15]:
pipeline = Pipeline([
    ('vect',CountVectorizer(tokenizer=tokenize)),
    ('tfidf',TfidfTransformer()),
    ('clf',MultiOutputClassifier(AdaBoostClassifier()))
])

In [16]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at 0x000001E47B5EB8B8>,
                                 vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMM

In [17]:
y_pred = pipeline.predict(X_test)

In [18]:
multioutput_f1score(y_test,y_pred)

0.9370848439946285

In [19]:
for i in range(len(category_names)):
    print('-'*60,'\n',"Category:", category_names[i],"\n", classification_report(y_test.iloc[:, i].values, y_pred[:, i]))
    print('Accuracy of',category_names[i], accuracy_score(y_test.iloc[:, i].values, y_pred[:,i]))

------------------------------------------------------------ 
 Category: related 
               precision    recall  f1-score   support

           0       0.62      0.12      0.20      1538
           1       0.78      0.98      0.87      4974
           2       0.62      0.12      0.20        42

    accuracy                           0.77      6554
   macro avg       0.67      0.41      0.42      6554
weighted avg       0.74      0.77      0.71      6554

Accuracy of related 0.7699115044247787
------------------------------------------------------------ 
 Category: request 
               precision    recall  f1-score   support

           0       0.91      0.96      0.93      5448
           1       0.74      0.51      0.61      1106

    accuracy                           0.89      6554
   macro avg       0.83      0.74      0.77      6554
weighted avg       0.88      0.89      0.88      6554

Accuracy of request 0.8881599023497101
------------------------------------------------

------------------------------------------------------------ 
 Category: infrastructure_related 
               precision    recall  f1-score   support

           0       0.94      0.99      0.97      6137
           1       0.43      0.13      0.20       417

    accuracy                           0.93      6554
   macro avg       0.69      0.56      0.58      6554
weighted avg       0.91      0.93      0.92      6554

Accuracy of infrastructure_related 0.9339334757400061
------------------------------------------------------------ 
 Category: transport 
               precision    recall  f1-score   support

           0       0.96      1.00      0.98      6261
           1       0.71      0.21      0.32       293

    accuracy                           0.96      6554
   macro avg       0.84      0.60      0.65      6554
weighted avg       0.95      0.96      0.95      6554

Accuracy of transport 0.9607873054623131
------------------------------------------------------------ 
 Categ

### 9. Export your model as a pickle file

In [20]:
pickle.dump(pipeline, open('../models/nb_classifier.pkl', "wb"))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.